In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import re

# Define a URL base onde o serviço Ollama está rodando localmente
base_url = 'http://localhost:11434'

# Escolhe qual modelo de linguagem será usado
model = 'llama3.2:3b'

# Inicializa o cliente do modelo Ollama com a URL base e o modelo escolhido
llm = ChatOllama(base_url=base_url, model=model)

# Define um template de mensagem do sistema, que funciona como uma instrução inicial para o modelo
system = SystemMessagePromptTemplate.from_template("""
Você é um assistente de IA útil que responde a perguntas de usuários relacionadas à análise de perfil do LinkedIn, atribuindo pontuações com base nas informações fornecidas, de acordo com os critérios estabelecidos abaixo.

Critérios de Pontuação:

- **Recomendações do último ano (20 pontos)**
- **Recomendações há mais de 2 anos (10 pontos)**
- **Título no LinkedIn preenchido (10 pontos)**
- **Resumo no perfil (Sobre) (25 pontos)**
- **Experiência profissional detalhada (20 pontos)**
- **Experiência recente com destaque de conquistas (15 pontos)**
- **Certificações recentes (15 pontos)**
- **Cursos concluídos nos últimos 2 anos (10 pontos)**
- **Conexões de alto nível (10 pontos)**
- **Participação em grupos ou comunidades relevantes (10 pontos)**
- **Publicações e Atividade (15 pontos)**
- **Endossos de habilidades (15 pontos)**
- **Idiomas adicionais (10 pontos)**
- **Projetos ou portfólio (20 pontos)**

Analise o perfil do LinkedIn com base nas informações fornecidas e forneça uma pontuação total (colocando "Pontuação Total: XXX/205, que é o máximo a se conseguir") junto com sugestões de melhorias para cada área. Seja detalhado e forneça uma lista de áreas que podem ser aprimoradas.
""")

# Função para fazer perguntas ao modelo de linguagem
def ask_llm(prompt):
    # Cria um template para a mensagem do usuário, contendo o prompt passado para a função
    prompt = HumanMessagePromptTemplate.from_template(prompt)
    
    # Combina a mensagem do sistema e do usuário numa lista de mensagens para o chat
    messages = [system, prompt]

    # Cria o template do chat com as mensagens do sistema e usuário
    template = ChatPromptTemplate(messages)

    # Constrói a cadeia de execução: envia o template para o modelo (llm) e depois usa um parser para obter a resposta como string simples
    qna_chain = template | llm | StrOutputParser()

    # Executa a cadeia (invoke) sem parâmetros adicionais e retorna a resposta do modelo
    return qna_chain.invoke({})

# Função para capturar e limpar o conteúdo de um perfil do LinkedIn usando Selenium
def get_linkedin_data():
    # Abrir o Chrome
    driver = webdriver.Chrome()

    # Com o Chrome aberto, acessar o link de login do linkedin
    driver.get("https://www.linkedin.com/login")
    email = driver.find_element(By.ID, 'username')
    email.send_keys(os.getenv('EMAIL'))
    password = driver.find_element(By.ID, 'password')
    password.send_keys(os.getenv('PASSWORD'))
    password.submit()

    url = os.getenv('LINKEDIN_URL')
    driver.get(url)

    # Captura o HTML da página
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Encontra a classe principal que mostra tudo do perfil
    profile = soup.find('main', {'class': 'AFoQvhgARpwZuvkYcojIRfUxjAGCtvmbnCCI'})

    # Trazer todas as sessões do perfil
    sections = profile.find_all('section', {'class': 'artdeco-card'})

    # Extrair texto de cada sessão
    sections_text = [section.get_text() for section in sections]

    # Limpar o texto extraído para remover duplicações e excessos
    def clean_text(text):
        text = re.sub(r'\n+', '\n', text) # Substitui repetições por uma única linha
        text = re.sub(r'\t+', '\t', text)
        text = re.sub(r'\t\s+', ' ', text)
        text = re.sub(r'\n\s+', '\n', text)
        return text

    sections_text = [clean_text(section) for section in sections_text]

    # Retornar o texto das seções limpo
    return sections_text

# Captura os dados do LinkedIn diretamente
sections_text = get_linkedin_data()

# Organiza os dados raspados em um formato adequado
profile_data = {
    "Nome e título": sections_text[0],  # Ajuste conforme a seção correta
    "Atividades e publicações": sections_text[1],  # Ajuste conforme a seção correta
    "Competências": sections_text[2],  # Ajuste conforme a seção correta
    "Recomendações": sections_text[3]  # Ajuste conforme a seção correta
}

# Template para análise do perfil
template = f"""
Analise o seguinte perfil do LinkedIn, atribua uma pontuação de acordo com os critérios mencionados acima e forneça sugestões para cada área com base na pontuação atribuída.

### Dados do perfil do LinkedIn:
Nome e título: {profile_data["Nome e título"]}
Atividades e publicações: {profile_data["Atividades e publicações"]}
Competências: {profile_data["Competências"]}
Recomendações: {profile_data["Recomendações"]}

### Dados extraídos:

1. Recomendações: Quantas são recentes? Atribua pontos e forneça sugestões.
2. Título: O título do perfil está claro e otimizado? Atribua pontos e forneça sugestões.
3. Resumo: O resumo está bem escrito e detalhado? Atribua pontos e forneça sugestões.
4. Experiência profissional: Detalhe sobre a experiência profissional. Atribua pontos e forneça sugestões.
5. Competências: A lista de competências é variada e relevante? Atribua pontos e forneça sugestões.
6. Outras áreas de melhorias (publicações, cursos, idiomas, etc.) e suas pontuações.

Com base nisso, forneça um feedback geral sobre como o perfil pode ser melhorado e pontue cada seção conforme a tabela de critérios."""

# Chama a função 'ask_llm' que envia o prompt ao modelo de linguagem e recebe a resposta
response = ask_llm(template)

# Exibe a resposta do modelo
print(response)


Aqui está uma análise detalhada do perfil do LinkedIn de Sarah Batagioti:

**Recomendações: 2**
Pontuação: 10 pontos (critério: Recomendações há mais de 2 anos)
Sugestões:
- Tente convidar novas pessoas para dar recomendações, como colegas de trabalho, supervisores ou clientes.
- Considere pedir recomendações de seus projetos específicos para dar mais profundidade às recomendações.

**Título: Desenvolvedora Full Stack | Python | JavaScript | HTML | CSS | React | MySQL**
Pontuação: 5 pontos (critério: Título no LinkedIn preenchido)
Sugestões:
- Considere otimizar o título do perfil para torná-lo mais específico e atraente para oportunidades de emprego.
- Tente incluir palavras-chave relevantes para o setor de desenvolvimento web.

**Resumo:**
Pontuação: 20 pontos (critério: Resumo no perfil)
Sugestões:
- Considere revisar e atualizar o resumo do perfil para refletir suas habilidades, experiência e interesses atuais.
- Tente manter a descrição curta, concisa e fácil de entender.

**Exper

In [2]:
import tkinter as tk
from tkinter import scrolledtext

# Função para exibir a resposta em uma janela
def show_response(response_text):
    # Cria a janela
    window = tk.Tk()
    window.title("Resposta do Modelo")
    window.geometry("600x400")  # Define o tamanho da janela

    # Cria um widget de texto para exibir a resposta
    text_area = scrolledtext.ScrolledText(window, wrap=tk.WORD, width=70, height=20)
    text_area.grid(column=0, row=0)
    text_area.insert(tk.INSERT, response_text)  # Insere a resposta do modelo
    text_area.config(state=tk.DISABLED)  # Torna o texto não editável

    # Exibe a janela
    window.mainloop()

# Supondo que 'response' seja a resposta obtida do modelo
response = ask_llm(template)

# Exibe a resposta na janela
show_response(response)
